# Importing data

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [3]:
# Percentage of spam data

def spam_percent():
    import numpy as np
    
    i = np.bincount(spam_data['target'])
    
    return i[1]*100/(i[0]+i[1])

spam_percent()

13.406317300789663

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def longest_token():
    
    vect = CountVectorizer().fit(X_train)
    
    return sorted(vect.get_feature_names_out(), key = len, reverse = True)[0]

longest_token()

'com1win150ppmx3age16subscription'

# First model

> Fitting and transforming the training data `X_train` using a Count Vectorizer with default parameters.

> Fitting a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`

> Area under the curve (AUC) score using the transformed test data

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def mul_NB():
    
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    auc_score = roc_auc_score(y_test, predictions) 
    
    return auc_score

mul_NB()

0.9720812182741116

# TF-IDF

> Fitting and transforming the training data `X_train` using a Tfidf Vectorizer with default parameters.

> 20 features have the smallest tf-idf and what 20 have the largest tf-idf

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf():
    
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    tfidf = X_train_vectorized.max(0).toarray()[0]
    index = tfidf.argsort()
    
    small_tfidf = np.array(vect.get_feature_names())[index[:20]]
    large_tfidf = np.array(vect.get_feature_names())[index[:-21:-1]]
    
    small_tfidf_series = pd.Series(sorted(tfidf)[:20], index = small_tfidf)
    large_tfidf_series = pd.Series(sorted(tfidf)[:-21:-1], index = large_tfidf)
    
    
    return (small_tfidf_series, large_tfidf_series)

tf_idf()

F:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

# Second model

> Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have document frequency lower than **3**.

> Fitting a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`

> Area under the curve (AUC) score using the transformed test data

In [7]:
def tfidf_mul_NB():
    
    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    auc_score = roc_auc_score(y_test, predictions) 
    
    return auc_score

tfidf_mul_NB()

0.9416243654822335

In [8]:
# Average length of documents for not spam and spam documents

def avg_len():
    
    spam_char = non_spam_char = 0
    
    for i in range(len(spam_data)):
        if spam_data.iloc[i]['target'] == 1:
            spam_char = spam_char + len(spam_data.iloc[i]['text'])
        else:
            non_spam_char = non_spam_char + len(spam_data.iloc[i]['text'])
        
    
    return (non_spam_char/np.bincount(spam_data['target'])[0], spam_char/np.bincount(spam_data['target'])[1])

avg_len()

(71.02362694300518, 138.8661311914324)

# Function to combine new feature to training data

In [9]:
def add_feature(X, feature_to_add):
    
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

# Third model

> Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have document frequency lower than **5**.

> Adding an additional feature, **the length of document**

> Fitting a Support Vector Classification model with regularization `C=10000`

> Area under the curve (AUC) score using the transformed test data

In [10]:
from sklearn.svm import SVC

def svc():
    
    from sklearn.svm import SVC
    
    def char_count(x):
        doc_len = []
        for i in x:
            doc_len.append(len(i))
        return doc_len
    
    train_doc_len = char_count(X_train)
    test_doc_len = char_count(X_test)
    
    vect = TfidfVectorizer(min_df = 5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    new_train_vectorized = add_feature(X_train_vectorized, train_doc_len)
    new_test_vectorized = add_feature(vect.transform(X_test), test_doc_len)
    
    model = SVC(C=10000).fit(new_train_vectorized, y_train)
    predictions = model.predict(new_test_vectorized)
    
    auc_score = roc_auc_score(y_test, predictions)

    return auc_score

svc()

0.9661689557407943

In [11]:
# Average number of digits per document for not spam and spam documents

def n_digits():
    
    spam_int = non_spam_int = 0
    
    import re
    
    for i in range(len(spam_data)):
        if spam_data.iloc[i]['target'] == 1:
            spam_int = spam_int + len(re.findall('[\d]', spam_data.iloc[i]['text']))
        else:
            non_spam_int = non_spam_int + len(re.findall('[\d]', spam_data.iloc[i]['text']))
        
    
    return (non_spam_int/np.bincount(spam_data['target'])[0], spam_int/np.bincount(spam_data['target'])[1])

n_digits()

(0.2992746113989637, 15.759036144578314)

# Fourth model

> Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have document frequency lower than **5** and using **word n-grams from n=1 to n=3**

> Adding an additional features, **the length of document**, **number of digits per document**

> Fitting a Logistic Regression model with regularization `C=100`

> Area under the curve (AUC) score using the transformed test data


In [12]:
from sklearn.linear_model import LogisticRegression

def lr():
    
    from sklearn.linear_model import LogisticRegression
    
    def char_count(x):
        doc_len = []
        for i in x:
            doc_len.append(len(i))
        return doc_len
    
    def int_count(x):
        import re
        count = 0
        int_num = []
        for i in x:
            int_num.append(len(re.findall('[\d]', i)))
        return int_num
    
    train_doc_len = char_count(X_train)
    test_doc_len = char_count(X_test)
    
    train_int_num = int_count(X_train)
    test_int_num = int_count(X_test)
    
    vect = TfidfVectorizer(min_df = 5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    new_train_vectorized1 = add_feature(X_train_vectorized, [train_doc_len, train_int_num])
    new_test_vectorized1 = add_feature(vect.transform(X_test), [test_doc_len, test_int_num])
    
#     new_train_vectorized2 = add_feature(new_train_vectorized1, train_int_num)
#     new_test_vectorized2 = add_feature(new_test_vectorized1, test_int_num)
    
    model = LogisticRegression(C=100).fit(new_train_vectorized1, y_train)
    predictions = model.predict(new_test_vectorized1)
    
    auc_score = roc_auc_score(y_test, predictions)
    
    
    return auc_score

lr()

F:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9759031798040846

In [13]:
# Average number of non-word characters per document for not spam and spam documents

def non_word_char():
    
    spam_non_word = non_spam_non_word = 0
    
    import re
    
    for i in range(len(spam_data)):
        if spam_data.iloc[i]['target'] == 1:
            spam_non_word = spam_non_word + len(re.findall('[\W]', spam_data.iloc[i]['text'])) 
        else:
            non_spam_non_word = non_spam_non_word + len(re.findall('[\W]', spam_data.iloc[i]['text']))
        
    
    return (non_spam_non_word/np.bincount(spam_data['target'])[0], spam_non_word/np.bincount(spam_data['target'])[1])

non_word_char()

(17.29181347150259, 29.041499330655956)

# Final model

> Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have document frequency lower than **5** and using **word n-grams from n=1 to n=5**

> Adding an additional features, **the length of document**, **number of digits per document**, **number of non-word characters**

> Fitting a Logistic Regression model with regularization `C=100`

> Area under the curve (AUC) score using the transformed test data


Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

In [14]:
def final_model():
    
    def char_count(x):
        doc_len = []
        for i in x:
            doc_len.append(len(i))
        return doc_len
    
    def int_count(x):
        import re
        count = 0
        int_num = []
        for i in x:
            int_num.append(len(re.findall('[\d]', i)))
        return int_num
    
    def non_word_count(x):
        import re
        count = 0
        non_word_num = []
        for i in x:
            non_word_num.append(len(re.findall('[\W]', i)))
        return non_word_num
    
    train_doc_len = char_count(X_train)
    test_doc_len = char_count(X_test)
    
    train_int_num = int_count(X_train)
    test_int_num = int_count(X_test)
    
    train_non_word_num = non_word_count(X_train)
    test_non_word_num = non_word_count(X_test)
    
    vect = CountVectorizer(min_df = 5, ngram_range=(2,5), analyzer= 'char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    new_train_vectorized1 = add_feature(X_train_vectorized, [train_doc_len, train_int_num, train_non_word_num])
    new_test_vectorized1 = add_feature(X_test_vectorized, [test_doc_len, test_int_num, test_non_word_num])
    
    model = LogisticRegression(C=100).fit(new_train_vectorized1, y_train)
    predictions = model.predict(new_test_vectorized1)
    
    auc_score = roc_auc_score(y_test, predictions)
    
    coeff_index = model.coef_[0].argsort()
    
    feature_names = vect.get_feature_names()
    new_feature_names = feature_names + ['length_of_doc', 'digit_count', 'non_word_char_count']
    
    small_coeff = np.array(new_feature_names)[coeff_index[:10]]
    large_coeff = np.array(new_feature_names)[coeff_index[:-11:-1]]
    
    return (auc_score, small_coeff, large_coeff)

final_model()

F:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
F:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(0.9813973821367333,
 array(['..', '. ', ' i', ' go', ' y', '? ', 'ok', 'pe', 'h ', 'go'],
       dtype='<U19'),
 array(['digit_count', 'ww', 'ne', 'co', 'ia', 'xt', 'ar', 'mob', ' ch',
        'uk'], dtype='<U19'))